## Bayes Net (or Belief Networks)


Bayesian networks use graphical structures to depict probabilistic relationships between a multitude of variables and facilitate probabilistic inference with these variables. The 1990s witnessed the development of outstanding algorithms for deriving Bayesian networks from observational data.

* We focus on intuitive strategies that emphasizes causal learning.

* We highlight the connections with the Bayesian approach through straightforward examples. This approach allows us to infer causal relationships from observational data.


Let's take this problem as our starting point or concrete example to introduce Bayesian Networks through a **graph collider**.

<img src="https://drive.google.com/uc?export=view&id=1ugh_UoUWoZ29sRmn9GoJTl0BeKlLq8f5" width="700">

Credits: https://discourse.pymc.io/t/bayes-nets-belief-networks-and-pymc/5150?page=2


### Bayes Network Using PyMC

In [3]:
import pymc as pm
import pytensor.tensor as pt
from pytensor import shared
import numpy as np

In [12]:
lookup_table = shared(np.asarray([
    [[.99, .01], [.1, .9]],
    [[.9, .1], [.1, .9]]]))

def f(smoker, covid):
    return lookup_table[smoker, covid]

with pm.Model() as m_bn:
    smoker = pm.Categorical('smoker', [.75, .25])
    covid = pm.Categorical('covid', [.9, .1])
    hospital = pm.Categorical('hospital', f(smoker, covid))
    prior_trace = pm.sample_prior_predictive(100000)


In [13]:
prior_covid = prior_trace.prior['covid'].values
prior_smoker = prior_trace.prior['smoker'].values
prior_hospital = prior_trace.prior['hospital'].values

In [14]:
predict_proba0 = prior_covid[(prior_smoker == 0) & (prior_hospital == 1)].mean()
predict_proba1 = prior_covid[(prior_smoker == 1) & (prior_hospital == 1)].mean()


In [15]:
print(f'P(covid|¬smoking, hospital) is {predict_proba0:1.2f}')
print(f'P(covid|smoking, hospital) is {predict_proba1:1.2f}')

P(covid|¬smoking, hospital) is 0.90
P(covid|smoking, hospital) is 0.50


### pgmpy Implementation

In [11]:
%pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 29.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [16]:
from pgmpy.models import BayesianNetwork

# Define the structure of the Bayesian Network
model = BayesianNetwork([
    ('Smoke', 'Hospital'),  # Edge from Smoke to Hospital
    ('Covid', 'Hospital')   # Edge from Covid to Hospital
])

In [17]:
# Define the Conditional Probability Distributions (CPDs)

from pgmpy.factors.discrete import TabularCPD

# CPD for Smoke
cpd_smoke = TabularCPD(variable='Smoke', variable_card=2, values=[[0.25], [0.75]], state_names={'Smoke': ['yes', 'no']})

# CPD for Covid
cpd_covid = TabularCPD(variable='Covid', variable_card=2, values=[[0.1], [0.9]], state_names={'Covid': ['yes', 'no']})

# CPD for Hospital
cpd_hospital = TabularCPD(variable='Hospital', variable_card=2,
                          values=[[0.9, 0.1, 0.9, 0.01],  # Probabilities for Hospital=yes
                                  [0.1, 0.9, 0.1, 0.99]], # Probabilities for Hospital=no
                          evidence=['Smoke', 'Covid'],
                          evidence_card=[2, 2],
                          state_names={'Hospital': ['yes', 'no'], 'Smoke': ['yes', 'no'], 'Covid': ['yes', 'no']})

# Add CPDs to the model
model.add_cpds(cpd_smoke, cpd_covid, cpd_hospital)

#-----------------#
#----- NOTES -----#
#-----------------#

# In the CPD for Hospital, the probabilities are ordered according to the Cartesian product of the evidence variables' states,
# i.e., ['Smoke_yes', 'Covid_yes'], ['Smoke_yes', 'Covid_no'], ['Smoke_no', 'Covid_yes'], and ['Smoke_no', 'Covid_no'].

# state_names parameter in each CPD is optional but helps make the output more interpretable.

# This example assumes binary states (yes/no) for simplicity, but pgmpy supports nodes with multiple states.

# The term evidence_card refers to the cardinality of the evidence variables. It specifies the number of states (or distinct values) that each evidence variable can take.


In [18]:
from pgmpy.inference import VariableElimination

# VariableElimination in pgmpy performs exact inference in Bayesian Networks
# by systematically eliminating variables, computing marginal distributions
# of interest or conditional distributions given evidence, optimizing
# computational efficiency through the order of elimination.

inference = VariableElimination(model)

# Probability of Hospitalization given Smoke=yes and Covid=yes
prob_hospital = inference.query(variables=['Hospital'], evidence={'Smoke': 'yes', 'Covid': 'yes'})
print(prob_hospital)


+---------------+-----------------+
| Hospital      |   phi(Hospital) |
+===============+=================+
| Hospital(yes) |          0.9000 |
+---------------+-----------------+
| Hospital(no)  |          0.1000 |
+---------------+-----------------+


In [19]:
# Probability of Covid given Smoke=yes and Hospital=yes
prob_covid = inference.query(variables=['Covid'], evidence={'Smoke': 'yes', 'Hospital': 'yes'})
print(prob_covid)

+------------+--------------+
| Covid      |   phi(Covid) |
+============+==============+
| Covid(yes) |       0.5000 |
+------------+--------------+
| Covid(no)  |       0.5000 |
+------------+--------------+
